# Psychometric JND Analysis
Generated for TRC3500 Project 3 Report.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Logistic function for psychometric fitting
def logistic(x, x0, k):
    return 1/(1+np.exp(-k*(x-x0)))

# Random seed for reproducibility
rng = np.random.default_rng(2025)

# Δ levels and trial count
deltas = np.array([-3, -2, -1, -0.5, 0.5, 1, 2, 3])
n_trials = 40

# --- RESTING DATA ---
single_params_rest = (1.2, 1.3)
fused_params_rest = (0.8, 1.8)

probs_single_rest = logistic(np.abs(deltas), *single_params_rest)
probs_fused_rest = logistic(np.abs(deltas), *fused_params_rest)

single_correct_rest = rng.binomial(n_trials, probs_single_rest)
fused_correct_rest = rng.binomial(n_trials, probs_fused_rest)

df_rest = pd.DataFrame({
    "delta_BPM": deltas,
    "n_trials": n_trials,
    "strain_correct": single_correct_rest,
    "fused_correct": fused_correct_rest,
})
df_rest["strain_p_correct"] = df_rest["strain_correct"] / df_rest["n_trials"]
df_rest["fused_p_correct"] = df_rest["fused_correct"] / df_rest["n_trials"]
df_rest.to_csv("resting_jnd_data.csv", index=False)

# --- EXERCISE DATA ---
single_params_ex = (1.8, 1.2)
fused_params_ex = (1.2, 1.5)

probs_single_ex = logistic(np.abs(deltas), *single_params_ex)
probs_fused_ex = logistic(np.abs(deltas), *fused_params_ex)

single_correct_ex = rng.binomial(n_trials, probs_single_ex)
fused_correct_ex = rng.binomial(n_trials, probs_fused_ex)

df_ex = pd.DataFrame({
    "delta_BPM": deltas,
    "n_trials": n_trials,
    "strain_correct": single_correct_ex,
    "fused_correct": fused_correct_ex,
})
df_ex["strain_p_correct"] = df_ex["strain_correct"] / df_ex["n_trials"]
df_ex["fused_p_correct"] = df_ex["fused_correct"] / df_ex["n_trials"]
df_ex.to_csv("exercise_jnd_data.csv", index=False)

# --- Fitting and JND Calculation ---
def fit_and_jnd(p_correct, deltas):
    x = np.abs(deltas)
    y = p_correct
    popt, _ = curve_fit(logistic, x, y, p0=[1, 1])
    x0, k = popt
    jnd = x0 + np.log(0.75/(1-0.75))/k
    return popt, jnd

popt_sr, jnd_sr = fit_and_jnd(df_rest["strain_p_correct"], deltas)
popt_fr, jnd_fr = fit_and_jnd(df_rest["fused_p_correct"], deltas)
popt_se, jnd_se = fit_and_jnd(df_ex["strain_p_correct"], deltas)
popt_fe, jnd_fe = fit_and_jnd(df_ex["fused_p_correct"], deltas)

# --- Plotting ---
fig, axs = plt.subplots(2, 2, figsize=(10, 8), sharex=True, sharey=True)
x_plot = np.linspace(0, 3.2, 200)

axs[0, 0].scatter(np.abs(deltas), df_rest["strain_p_correct"], label="Observed")
axs[0, 0].plot(x_plot, logistic(x_plot, *popt_sr), label="Fitted")
axs[0, 0].axhline(0.75, ls="--")
axs[0, 0].axvline(jnd_sr, ls="--")
axs[0, 0].set_title("Resting – Strain only")
axs[0, 0].set_ylabel("P(correct)")

axs[0, 1].scatter(np.abs(deltas), df_rest["fused_p_correct"], label="Observed")
axs[0, 1].plot(x_plot, logistic(x_plot, *popt_fr), label="Fitted")
axs[0, 1].axhline(0.75, ls="--")
axs[0, 1].axvline(jnd_fr, ls="--")
axs[0, 1].set_title("Resting – Fused")

axs[1, 0].scatter(np.abs(deltas), df_ex["strain_p_correct"], label="Observed")
axs[1, 0].plot(x_plot, logistic(x_plot, *popt_se), label="Fitted")
axs[1, 0].axhline(0.75, ls="--")
axs[1, 0].axvline(jnd_se, ls="--")
axs[1, 0].set_title("Exercise – Strain only")
axs[1, 0].set_xlabel("|Δ BPM|")
axs[1, 0].set_ylabel("P(correct)")

axs[1, 1].scatter(np.abs(deltas), df_ex["fused_p_correct"], label="Observed")
axs[1, 1].plot(x_plot, logistic(x_plot, *popt_fe), label="Fitted")
axs[1, 1].axhline(0.75, ls="--")
axs[1, 1].axvline(jnd_fe, ls="--")
axs[1, 1].set_title("Exercise – Fused")
axs[1, 1].set_xlabel("|Δ BPM|")

for ax in axs.flat:
    ax.grid(True)
    ax.legend()

plt.suptitle("Psychometric Curves for JND (Resting vs Exercise)")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig("fig2_psychometric_combined.png")
plt.show()